In [1]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model, Sequential
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from math import *
import os

In [0]:
def deep_autoencoder(input_dim, encoded_dim, x_train, x_test, my_image_test):
    autoencoder = Sequential()

    # Encoder Layers
    autoencoder.add(Dense(4 * encoded_dim, input_shape=(input_dim,), activation='relu'))
    autoencoder.add(Dense(2 * encoded_dim, activation='relu'))
    autoencoder.add(Dense(encoded_dim, activation='relu'))

    # Decoder Layers
    autoencoder.add(Dense(2 * encoded_dim, activation='relu'))
    autoencoder.add(Dense(4 * encoded_dim, activation='relu'))
    autoencoder.add(Dense(input_dim, activation='sigmoid'))
            
    input_img = Input(shape=(input_dim,))
    encoder_layer1 = autoencoder.layers[0]
    encoder_layer2 = autoencoder.layers[1]
    encoder_layer3 = autoencoder.layers[2]
    encoder = Model(input_img, encoder_layer3(encoder_layer2(encoder_layer1(input_img))))
    
    autoencoder.summary()
    encoder.summary()
    
    autoencoder.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    autoencoder.fit(x_train, x_train,
                epochs=3500,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test, x_test))
    
    encoded_imgs = encoder.predict(x_test)
    decoded_imgs = autoencoder.predict(x_test)
    
    return (encoded_imgs, decoded_imgs)

In [0]:
DATADIR = './data'
#CATEGORIES = ['test', 'train']

In [0]:
for img in os.listdir(DATADIR):
    try:      
        # my_image_original = mpimg.imread(os.path.join(path, img))
        my_image_original = mpimg.imread((os.path.join(DATADIR, img)))
        max_original_value = float(my_image_original.max())
        x_original = my_image_original.astype('float32') / max_original_value
        x_original = x_original.reshape((len(x_original), np.prod(x_original.shape[1:])))

        # my_image_test = mpimg.imread(os.path.join(path, img))
        # max_train_test = float(my_image_train.max())
        # x_test = my_image_test.astype('float32') / max_train_test
        # x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
    
        (encoded_imgs, decoded_imgs) = deep_autoencoder(x_original.shape[1], 64, x_original, x_original, my_image_original)  
        
        to_reshape = my_image_original.shape
        x_original_result = x_original.reshape(to_reshape[0], to_reshape[1], to_reshape[2])
        decoded_imgs_result = decoded_imgs.reshape(to_reshape[0], to_reshape[1], to_reshape[2])      
        # x_test_result = x_original.reshape(to_reshape[0], to_reshape[1], to_reshape[2])


        plt.axis("off")

        plt.figure()
        # plt.subplot(3, 1, img+1)
        imgplot_original_result = plt.imshow(x_original_result)
        plt.title('Train image')
        mpimg.imsave('./result/' + img + '_original.png', my_image_original)

        plt.figure()
        # plt.subplot(3, 1, img+1)
        imgplot_encoded_result = plt.imshow(encoded_imgs)
        plt.title('Encoded image')
        mpimg.imsave('./result/' + img + '_encoded_imgs.png', encoded_imgs)

        plt.figure()
        # plt.subplot(3, 1, img+1)
        imgplot_decoded_imgs = plt.imshow(decoded_imgs_result)
        plt.title('Decoded image')
        mpimg.imsave('./result/' + img + '_decoded_imgs.png', decoded_imgs_result)

    except Exception as e:
        pass

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               6145280   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_5 (Dense)              (None, 24004)             6169028   
Total params: 12,396,804
Trainable params